In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from src.qaoa.models.MaxCutProblem import MaxCutProblem

def visualize(data, x,y, hues, chart):
    fig = plt.figure(figsize=(8,6))
    match chart:
        case 'line':
            chart_type = sns.lineplot
        case 'bar':
            chart_type = sns.barplot
        case 'box':
            chart_type = sns.boxplot
        case None:
            chart_type = sns.plot

    # make a copy to avoid modifying the original DataFrame and avoid warnings
    data_to_plot = data.copy()

    try:
        if len(hues) > 1:
            data_to_plot['hues'] = data_to_plot[hues].astype(str).agg('-'.join, axis=1)
            _ = chart_type(data=data_to_plot, x=x, y=y, hue='hues', palette='viridis')
            plt.legend(title=f'{y} for {x}, sorted by {[h + ", " for h in hues]}', bbox_to_anchor=(1.05, 1), loc='upper left')
        elif len(hues) == 1:
            data_to_plot['hues'] = data_to_plot[hues[0]]
            _ = chart_type(data=data_to_plot, x=x, y=y, hue='hues', palette='viridis')
            plt.legend(title=f'{y} for {x}, sorted by {[h + ", " for h in hues]}', bbox_to_anchor=(1.05, 1), loc='upper left')
        else:
            _ = chart_type(data=data_to_plot, x=x, y=y)
            plt.legend(title=f'{y} for {x}, sorted by {[h + ", " for h in hues]}', bbox_to_anchor=(1.05, 1), loc='upper left')
            #plt.show()

            plt.axhline(y=1, color='red', linestyle='--', linewidth=2)

        plt.axhline(y=1, color='red', linestyle='--', linewidth=2)
        plt.show()
        plt.close(fig)
    except (KeyError, ValueError) as e:
        print(f"You've passed an incorrect column name.\n The correct ones are: \n{data_to_plot.columns}\nException: {e}")


In [ ]:
import sqlite3
import pandas as pd

# read from database 
# Connect to the SQLite database
results = pd.read_sql_query("SELECT * FROM runs", sqlite3.connect('qruns.db'))
conn = sqlite3.connect('qruns.db')



results_lagrange = results[((results['qaoa_variant'] == 'multiangle') & (results['depth'] == 1)) |
              ((results['qaoa_variant'] == 'vanilla') & (results['depth'] == 4))]
#problem: lagrange 2 is with a lot more data points and configuration settings.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
df = results.copy()


df = df[df['warm_start'] == False]
df = df[df['infeasible'] == False]

fig, axes = plt.subplots(1, 2, figsize=(16, 6), constrained_layout=True)

# First subplot - Quantum Function Evaluations
sns.boxplot(data=df, x="depth", y="quantum_func_evals", hue="qaoa_variant", ax=axes[0])
axes[0].set_xlabel("QAOA Variant", fontsize=20)
axes[0].set_ylabel("Number of Function Evaluations", fontsize=20)
axes[0].tick_params(axis='y', labelsize=20)
axes[0].tick_params(axis='x', labelsize=20)
axes[0].legend(title="Depth", title_fontsize=16, fontsize=14)
axes[0].grid(True, axis='y', linestyle='--', alpha=0.6)
axes[0].set_xticklabels([lbl.get_text().capitalize() for lbl in axes[0].get_xticklabels()])

# Second subplot - Solution Quality
sns.barplot(data=df, x="qaoa_variant", y="ratio", hue="depth", ax=axes[1])
axes[1].set_xlabel("QAOA Variant", fontsize=20)
axes[1].set_ylabel("Ratio (Solution/Optimal)", fontsize=20)
axes[1].tick_params(axis='y', labelsize=20)
axes[1].tick_params(axis='x', labelsize=20)
axes[1].legend(title="Depth", title_fontsize=16, fontsize=14)
axes[1].grid(True, axis='y', linestyle='--', alpha=0.6)
axes[1].set_xticklabels([lbl.get_text().capitalize() for lbl in axes[1].get_xticklabels()])

# Add a horizontal red line at y=1 for the ratio plot to indicate optimal solution
axes[1].axhline(y=1, color='red', linestyle='--', linewidth=2)

plt.savefig('CS_metrics.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:

# Create a copy of results to work withd
df = results.copy()
df = df[df['infeasible'] == False]# Get unique graph names
graph_names = df['graph_name'].unique()
df = df[df['warm_start']==False]


# Now create a plot showing function evaluations as well
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, graph in enumerate(graph_names):
    # Filter data for this graph
    graph_df = df[df['graph_name'] == graph]
    
    # Get graph size for the title
    graph_size = graph_df['graph_size'].iloc[0]
    
    # Create barplot for function evaluations
    sns.barplot(data=graph_df, x="qaoa_variant", y="ratio", hue="depth", ax=axes[i])
    
    # Customize plot
    axes[i].set_title(f"Graph Size {graph_size} ", fontsize=20)
    axes[i].set_ylabel("Number of Function Evaluations", fontsize=20)
    axes[i].tick_params(axis='both', labelsize=20)
    axes[i].axhline(y=1, color='red', linestyle='--', linewidth=2)
    # Adjust legend
    handles, labels = axes[i].get_legend_handles_labels()
    axes[i].set(xlabel=None)
    axes[i].legend(handles, labels, title="Depth", title_fontsize=16, fontsize=14, loc="lower right")
    axes[i].set_xticklabels([lbl.get_text().capitalize() for lbl in axes[i].get_xticklabels()])
axes[1].set_title(f"Graph Size {9} Dense", fontsize=20)
axes[2].set_title(f"Graph Size {9} Sparse", fontsize=20)
plt.tight_layout()
fig.subplots_adjust(top=0.93)  # Make room for the suptitle

# Save figure
plt.savefig('CS_allgraphs_comparison.pdf', bbox_inches='tight')

plt.show()


In [ ]:
import seaborn as sns
import pandas as pd

import matplotlib.pyplot as plt

# Filter only graph size 6 data
df_size_6 = results[results['graph_size'] == 6].copy()
df_size_6 = df_size_6[df_size_6['warm_start']==False]
#df_size_6 = df_size_6[df_size_6['infeasible']==False]
# Create figure
plt.figure(figsize=(10, 6))

# Set theme
sns.set_theme(style="whitegrid", palette="colorblind")

# Plot data
sns.barplot(data=df_size_6, x="qaoa_variant", y="ratio", hue="depth")

# Add horizontal line at ratio=1
plt.axhline(y=1, color='red', linestyle='--', linewidth=2)

# Customize plot
plt.title("Solution Quality for Graph Size 6 (Including Infeasible Solutions)", fontsize=14)
plt.xlabel("QAOA Variant", fontsize=14)
plt.ylabel("Ratio (Solution/Optimal)", fontsize=14)
plt.legend(title="Depth", title_fontsize=14, fontsize=12)
plt.grid(True, axis='y', linestyle='--', alpha=0.6)
ax = plt.gca()
ax.set_xticklabels([label.get_text().capitalize() for label in ax.get_xticklabels()], fontsize=14)



plt.tight_layout()
#plt.savefig('CS_infeasible_smallgraph.pdf',bbox_inches='tight')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# --- Data prep ---
df_analysis = results.copy()
print(results.groupby(['graph_name'])['infeasible'].value_counts())
df_analysis['infeasible_bool'] = df_analysis['infeasible'].astype(str) == 'True'
df_analysis = df_analysis[df_analysis['warm_start']==False]
# Group by config and count
grouped = df_analysis.groupby(['qaoa_variant', 'depth',  'infeasible_bool']).size().reset_index(name='count')

# Pivot to wide format
pivot_df = grouped.pivot_table(
    index=['qaoa_variant', 'depth', ],
    columns='infeasible_bool',
    values='count',
    fill_value=0
).reset_index()

# Rename for clarity
pivot_df.rename(columns={False: 'feasible', True: 'infeasible'}, inplace=True)

# Calculate percentages
pivot_df['total'] = pivot_df['feasible'] + pivot_df['infeasible']
pivot_df['feasible_pct'] = pivot_df['feasible'] / pivot_df['total'] * 100
pivot_df['infeasible_pct'] = pivot_df['infeasible'] / pivot_df['total'] * 100

# Custom labels for x-axis
pivot_df['label'] = pivot_df.apply(lambda row: f"Depth {row['depth']}, {row['qaoa_variant'].capitalize()}", axis=1)

# --- Plotting ---
sns.set_theme(style="whitegrid", context="talk")
colors = {'Feasible': '#3D5B8D', 'Infeasible': '#9D3E42'}

fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(pivot_df))
width = 0.6

# Plot feasible bar base
bars1 = ax.bar(x, pivot_df['feasible_pct'], width, label='Feasible', color=colors['Feasible'])

# Plot infeasible on top
bars2 = ax.bar(x, pivot_df['infeasible_pct'], width, bottom=pivot_df['feasible_pct'], 
               label='Infeasible', color=colors['Infeasible'])

# Add percentage labels
for i, row in pivot_df.iterrows():
    if row['feasible_pct'] > 0:
        ax.text(x[i], row['feasible_pct'] / 2, f"{row['feasible_pct']:.0f}%", ha='center', va='center', fontsize=10, color='white')
    if row['infeasible_pct'] > 0:
        ax.text(x[i], row['feasible_pct'] + row['infeasible_pct'] / 2, f"{row['infeasible_pct']:.0f}%", ha='center', va='center', fontsize=10, color='black')
# Axes and title
ax.set_xticks(x)
ax.set_xticklabels(pivot_df['label'], rotation=30, ha='right', fontsize=14)
ax.set_ylabel("Percentage of Solutions", fontsize=13)
ax.set_xlabel("QAOA Configuration", fontsize=13)
ax.set_ylim(0, 100)

# Red dashed baseline at 100% (optional)

# Legend and style
ax.legend(title="Solution Type", fontsize=11, title_fontsize=12, loc="lower right")
sns.despine()
plt.tight_layout()
plt.savefig('CS_feasible_percent.pdf', bbox_inches='tight')

plt.show()


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

# Filter to only warm-started entries
df_ws = results.copy()  # [results['warm_start'] == True]

df_ws = df_ws[((df_ws['qaoa_variant'] == 'multiangle') & (df_ws['depth'] == 1)) |
              ((df_ws['qaoa_variant'] == 'vanilla') & (df_ws['depth'] == 4))]

# Cleaned labels
graph_label_map = {
    '>>graph6<<Emz_': 'Graph 6',
    '>>graph6<<HmzffJz': 'Graph 9 (Dense)',
    '>>graph6<<Hh_iS_u': 'Graph 9 (Sparse)',
    '>>graph6<<KmzffJznl{hU': 'Graph 12',
}
df_ws['graph_label'] = df_ws['graph_name'].map(graph_label_map)

label_map = {
    'False0': 'Equal Superposition',
    'True0': 'WS HD: 0',
    'True1': 'WS HD: 1',
    'True3': 'WS HD: 3',
    'True5': 'WS HD: 5',
}
df_ws['Qubit Initialization'] = df_ws['warm_start'].astype(str) + df_ws['hamming_dist'].astype(str)
df_ws['Qubit Initialization'] = df_ws['Qubit Initialization'].map(label_map)

# Group by hamming distance to compute percentage of infeasible runs
infeasible_pct = (
    df_ws.groupby('Qubit Initialization')['infeasible']
    .mean()
    .reset_index(name='perc_infeasible')
)
infeasible_pct['perc_infeasible'] *= 100

# Compute percent improvement from the warm-started string.
# 100 * (quantum_obj_value - ws_value) / quantum_obj_value.
df_ws = df_ws[df_ws['quantum_obj_value'] != 0].copy()
df_ws['ws_value'] = df_ws['ws_value'].astype(float)  # ensure numeric
df_ws['perc_improvement'] = np.where(
    df_ws['ws_value'].notnull(),
    100 * (df_ws['ws_value'] - df_ws['quantum_obj_value']) / df_ws['ws_value'],
    np.nan
)
df_ws['actual_improvement'] = np.where(
    df_ws['ws_value'].notnull(),
    df_ws['ws_value'] - df_ws['quantum_obj_value'],
    np.nan
)

# Create a 2x1 figure (2 rows, 1 column)
fig, axes = plt.subplots(1, 2, figsize=(24, 10))

# 1) Runtime (time_elapsed) by hamming distance
sns.kdeplot(data=df_ws, x='time_elapsed', ax=axes[0], hue='Qubit Initialization', palette='viridis', linewidth=3,legend='full')
axes[0].set_title('Runtime by Qubit Initialization', fontsize=28)
axes[0].set_ylabel('Density', fontsize=26)
axes[0].set_xlabel('Time Elapsed (s)', fontsize=26)
axes[0].set_xlim(left=0)
axes[0].tick_params(axis='both', labelsize=22)
leg = axes[0].get_legend()                     # legend Seaborn already created
leg.set_title("Qubit Initialization", prop={"size": 24})
for txt in leg.get_texts():
    txt.set_fontsize(22)
df_ws['ws_ratio'] = df_ws['ws_value']/df_ws['classic_value']

# 2) Ratio distribution by hamming distance — upgraded
sns.set(style="whitegrid", context="talk")
sns.lineplot(
    data=df_ws,
    x='Qubit Initialization',
    y='ratio',
    ax=axes[1],
    marker='o',
    markersize=12,
    linewidth=3,
    color='C0',
    errorbar=None,
    hue='graph_label'
)
axes[1].set_title('Average Ratio by Qubit Initialization', fontsize=28)
axes[1].set_ylabel('Ratio', fontsize=26)
axes[1].set_xlabel('Qubit Initialization', fontsize=26)
axes[1].tick_params(axis='both', labelsize=22)
axes[1].legend(title='Graph Type', title_fontsize=24, fontsize=22)

plt.tight_layout()
plt.savefig('ws_stats.pdf', bbox_inches='tight')
plt.show()

# Create one plot with two subplots side by side for multiangle and vanilla QAOA variants
fig, axarr = plt.subplots(1, 2, figsize=(24, 8))
sns.set_style("whitegrid")

for ax, qaoa_type in zip(axarr, ['multiangle', 'vanilla']):
    # Filter data for the current QAOA variant and warm-started solutions with hamming distance > 0
    current_df = df_ws[df_ws['qaoa_variant'] == qaoa_type].copy()
    ws_true = current_df[(current_df['warm_start'] == True) & (current_df['hamming_dist'] > 0)].copy()
    ws_true = ws_true.sort_values('graph_name')
    
    # Create a scatterplot on the current axis
    sns.scatterplot(
        data=ws_true,
        x='ws_value',
        y='quantum_obj_value',
        hue='graph_label',
        style='Qubit Initialization',
        s=100,
        ax=ax
    )
    
    # Draw a diagonal line (y = x) to show the line of no improvement
    lims = [
        min(ax.get_xlim()[0], ax.get_ylim()[0]),
        max(ax.get_xlim()[1], ax.get_ylim()[1])
    ]
    ax.plot(lims, lims, 'r--', label='No Improvement Line')
    
    ax.set_title(f'{qaoa_type.capitalize()} QAOA: Warm Start Value vs Quantum Solution Value', fontsize=20)
    ax.set_xlabel('Warm Start Value', fontsize=16)
    ax.set_ylabel('Quantum Solution Value', fontsize=16)
    ax.legend(title='Graph Label', fontsize=12)
    
plt.tight_layout()
plt.savefig('warm_start_scatter_side_by_side.pdf', bbox_inches='tight')
plt.show()

# Add a diagonal line (y=x) to show where quantum solution = warm start
lims = [
    min(ax.get_xlim()[0], ax.get_ylim()[0]),
    max(ax.get_xlim()[1], ax.get_ylim()[1])
]
plt.plot(lims, lims, 'r--', label='No Improvement Line')

# Customize plot
plt.title('Warm Start Value vs Quantum Solution Quality', fontsize=15)
plt.xlabel('Warm Start Solution Value', fontsize=16)
plt.ylabel('Quantum Solution Value', fontsize=16)
plt.grid(True, alpha=0.3)
plt.legend(title='Graph Type', title_fontsize=16)



plt.tight_layout()
plt.savefig('warm_start_performance.pdf', bbox_inches='tight')
plt.show()
df_ws


# Re-plot average ratio with ws_value overlaid as dotted lines
plt.figure(figsize=(10, 6))

# 2) Ratio distribution by hamming distance — upgraded
sns.set(style="whitegrid", context="talk")
ax = sns.lineplot(
    data=df_ws,
    x='Qubit Initialization',
    y='ws_ratio',
    marker='o',
    markersize=12,
    linewidth=3,
    color='C0',
    errorbar=None,
    hue='graph_label'
)
ax.set_title('WS String Ratio by Qubit Initialization', fontsize=28)
ax.set_ylabel('WS String Ratio', fontsize=26)
ax.set_xlabel('Qubit Initialization', fontsize=26)
ax.tick_params(axis='both', labelsize=22)
ax.legend(title='Graph Type', title_fontsize=24, fontsize=22)
plt.tight_layout()
plt.savefig('ws_string_ratio.pdf', bbox_inches='tight')
plt.show()

# Re-plot average ratio with ws_value overlaid as dotted lines
plt.figure(figsize=(10, 6))

# 2) Ratio distribution by hamming distance — upgraded
sns.set(style="whitegrid", context="talk")
ax = sns.lineplot(
    data=df_ws,
    x='Qubit Initialization',
    y='percent_measure_optimal',
    marker='o',
    markersize=12,
    linewidth=3,
    color='C0',
    errorbar=None,
    hue='graph_label'
)
ax.set_title('Probability of measuring Optimal by Qubit Initialization', fontsize=28)
ax.set_ylabel('Probability', fontsize=26)
ax.set_xlabel('Qubit Initialization', fontsize=26)
ax.tick_params(axis='both', labelsize=22)
ax.legend(title='Graph Type', title_fontsize=24, fontsize=22)
plt.tight_layout()
plt.savefig('perc_measure_optimal_HD.pdf', bbox_inches='tight')
plt.show()

print("Lagrangian multipliers used:", df_ws['lagrangian_multiplier'].unique())

In [ ]:
df_ws = results.copy()
df_ws['actual_improvement'] = np.where(
    df_ws['ws_value'].notnull(),
    df_ws['ws_value'] - df_ws['quantum_obj_value'],
    np.nan
)
df_ws = df_ws[((df_ws['qaoa_variant'] == 'multiangle') & (df_ws['depth'] == 1)) |
              ((df_ws['qaoa_variant'] == 'vanilla') & (df_ws['depth'] == 4))]

# Cleaned labels
graph_label_map = {
    '>>graph6<<Emz_': 'Graph 6',
    '>>graph6<<HmzffJz': 'Graph 9 (Dense)',
    '>>graph6<<Hh_iS_u': 'Graph 9 (Sparse)',
    '>>graph6<<KmzffJznl{hU': 'Graph 12',
}
df_ws['graph_label'] = df_ws['graph_name'].map(graph_label_map)

label_map = {
    'False0': 'Equal Superposition',
    'True0': 'WS HD: 0',
    'True1': 'WS HD: 1',
    'True3': 'WS HD: 3',
    'True5': 'WS HD: 5',
}
df_ws['Qubit Initialization'] = df_ws['warm_start'].astype(str) + df_ws['hamming_dist'].astype(str)
df_ws['Qubit Initialization'] = df_ws['Qubit Initialization'].map(label_map)
# Create a new dataframe to analyze improvement statistics
improvement_stats = pd.DataFrame(columns=['qaoa_variant', 'Qubit Initialization', 'Improved %', 'Same %', 'Worsened %', 'Count'])

# Include all warm started data including hamming distance 0
ws_all = df_ws.copy()

# Filter out Equal Superposition for the visualization
ws_all_filtered = ws_all[ws_all['Qubit Initialization'] != 'Equal Superposition']

# Create a figure for both variants together
plt.figure(figsize=(16, 8))

# For each initialization type and QAOA variant, compute improvement statistics
for qaoa_var in ['multiangle', 'vanilla']:
    ws_filtered = ws_all[ws_all['qaoa_variant'] == qaoa_var]
    
    for init_type in ws_filtered['Qubit Initialization'].unique():
        subset = ws_filtered[ws_filtered['Qubit Initialization'] == init_type]
        total = len(subset)
        
        if total > 0:  # Only process if we have data for this combination
            # Calculate percentages
            improved = (subset['actual_improvement'] > 0.0001).sum() / total * 100
            same = (abs(subset['actual_improvement']) <= 0.0001).sum() / total * 100 
            worsened = (subset['actual_improvement'] < -0.0001).sum() / total * 100
            
            # Add to results dataframe
            new_row = pd.DataFrame({
                'qaoa_variant': [qaoa_var],
                'Qubit Initialization': [init_type],
                'Improved %': [improved],
                'Same %': [same], 
                'Worsened %': [worsened],
                'Count': [total]
            })
            improvement_stats = pd.concat([improvement_stats, new_row], ignore_index=True)

# Sort by initialization type and variant
improvement_stats = improvement_stats.sort_values(['qaoa_variant', 'Qubit Initialization'])

# Create a nice visualization with both variants side by side
# Filter out Equal Superposition for the visualization
improvement_stats_filtered = improvement_stats[improvement_stats['Qubit Initialization'] != 'Equal Superposition']
improvement_stats_melted = pd.melt(
    improvement_stats_filtered, 
    id_vars=['qaoa_variant', 'Qubit Initialization', 'Count'],
    value_vars=['Improved %', 'Same %', 'Worsened %'],
    var_name='Result', value_name='Percentage'
)

# Set up a custom color palette
palette = {'Improved %': '#2ca02c', 'Same %': '#1f77b4', 'Worsened %': '#d62728'}

# Create a grouped bar chart with both variants side by side
ax = sns.catplot(
    x='Qubit Initialization', 
    y='Percentage', 
    hue='Result', 
    col='qaoa_variant',
    data=improvement_stats_melted, 
    kind='bar',
    palette=palette,
    height=6, aspect=1.2
)
# Clean up extra legends

sns.move_legend(ax, 'lower left', bbox_to_anchor= (0.33,-0.1),ncol=3,title=None)
# Set titles and labels
ax.set_axis_labels(None, 'Percentage of Solutions')
ax.set_titles(col_template='{col_name} QAOA')

# Adjust ylim
for ax_i in ax.axes[0]:
    ax_i.set_ylim(0, 100)
    ax_i.grid(axis='y', alpha=0.3)
    # Add percentage values on top of each bar
    for p in ax_i.patches:
        height = p.get_height()
        if height > 0:
            ax_i.text(
                p.get_x() + p.get_width()/2.,
                height + 1,
                f'{height:.1f}%',
                ha='center', 
                fontsize=9
            )
    
    # Clean up x-axis labels
    ax_i.set_xticklabels(ax_i.get_xticklabels(), rotation=45, ha='right')

# Capitalize first letter of each subplot title
for ax_i, title in zip(ax.axes[0], ['Multiangle', 'Vanilla']):
    ax_i.set_title(f"{title} QAOA")

plt.tight_layout()
plt.savefig('warm_start_improvement_stats_combined.pdf', bbox_inches='tight')
plt.show()

# Display the detailed table
display(improvement_stats.round(2))


# Create a figure and analyze runtime performance
plt.figure(figsize=(14, 8))

# Plot runtime distribution by QAOA variant
sns.kdeplot(data=df_ws, x='quantum_func_evals', hue='qaoa_variant', palette='viridis', linewidth=3)

# Set plot styling
plt.title('Runtime by QAOA Variant', fontsize=16)
plt.ylabel('Density', fontsize=14)
plt.xlabel('Number of Function Evaluations', fontsize=14)
plt.xlim((0,5000))
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('runtime_by_qaoa_variant.pdf', bbox_inches='tight')
plt.show()

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid", palette="colorblind")
# Create a visualization to analyze solution ratio by lagrangian multiplier and qaoa variant
plt.figure(figsize=(12, 6))



# Filter only feasible solutions
feasible_df = results_lagrange[results_lagrange['infeasible'] == False].copy()
feasible_df = feasible_df[feasible_df['warm_start']==False]
#feasible_df = results_lagrange.copy()
# For better analysis, let's group the data by variant, lagrangian multiplier and depth
# This will give us a clearer picture of how these factors affect solution quality



# Focus on a subset with specific depths that provide a good comparison point
# Keep only depth 1 for multiangle and depth 4 for vanilla for fair comparison
subset = feasible_df[
    ((feasible_df['qaoa_variant'] == 'multiangle') & (feasible_df['depth'] == 1)) | 
    ((feasible_df['qaoa_variant'] == 'vanilla') & (feasible_df['depth'] == 4))
]

# Let's analyze what depth & variants are available in our dataset
print("Distribution of depth and variants in the dataset:")
print(subset.groupby(['qaoa_variant', 'depth','lagrangian_multiplier']).size())
# Create a new column that combines variant and depth for better labeling
subset['QAOA Implementation'] = subset.apply(lambda row: f"{row['qaoa_variant'].capitalize()} and Depth: {row['depth']}", axis=1)

# Create the barplot
sns.barplot(data=subset, x='lagrangian_multiplier', y='percent_measure_optimal', hue='QAOA Implementation', palette='viridis')


# Add horizontal line at ratio=1 to indicate optimal solution quality
plt.axhline(y=1, color='red', linestyle='--', linewidth=2)

# Customize the plot
plt.title('Probability of measuring Optimal Solution by Lagrangian Multiplier and QAOA Variant ', fontsize=14)
plt.xlabel('Lagrangian Multiplier', fontsize=12)
plt.ylabel('Probability of measuring Optimal Solution', fontsize=12)
plt.legend(title='QAOA Variant and Depth', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.savefig('lagrangian_percentoptimal_depth1_4.pdf', bbox_inches='tight')

plt.tight_layout()
plt.show()


sns.kdeplot(data= subset, x = 'quantum_func_evals' , hue='QAOA Implementation', multiple='stack',palette='viridis')
plt.title('KDE-plot of Quantum Function Evaluation values per QAOA Variant ', fontsize=14)
sns.despine()
plt.xlabel('Quantum Function Evaluations', fontsize= 12)
plt.savefig('lagrangian_kdeplot.pdf', bbox_inches='tight')

# Create a figure for quantum function evaluations by Lagrangian multiplier
plt.figure(figsize=(12, 6))

# Create the boxplot to show distribution
sns.barplot(
    data=feasible_df, 
    x='lagrangian_multiplier', 
    y='quantum_func_evals',
    palette='viridis'
)

# Customize the plot
plt.title('Quantum Function Evaluations by Lagrangian Multiplier', fontsize=14)
plt.xlabel('Lagrangian Multiplier', fontsize=12)
plt.ylabel('Number of Function Evaluations', fontsize=12)
plt.legend(title='QAOA Variant and Depth', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('lagrangian_func_evals.pdf', bbox_inches='tight')
plt.show()

#
plt.figure(figsize=(12, 6))


sns.barplot(
    data=feasible_df,
    x='lagrangian_multiplier',
    y='time_elapsed',
    palette='viridis'
)


plt.title('Runtime by Lagrangian Multiplier', fontsize=14)# Customize the plotntation',tiplier',    data=subset, sns.barplot( Create a figure for runtime by Lagrangian multiplier# Create a figure for quantum function evaluations by Lagrangian multiplier
plt.figure(figsize=(12, 6))
print(results_lagrange.groupby(['lagrangian_multiplier','hamming_dist','warm_start'])['ratio'].mean())
# Create the boxplot to show distribution
sns.lineplot(
    data=subset, 
    x='lagrangian_multiplier', 
    y='ratio',
    hue='QAOA Implementation',
    palette='viridis',
    errorbar=None,
    marker='o',
    markersize=8,
)
red_dot = results_lagrange[(results_lagrange['lagrangian_multiplier'] == 1) & (results_lagrange['depth']==4) & (results_lagrange['warm_start']==False)]['ratio'].mean()
print('red dot', red_dot)
# Customize the plot
plt.title('Ratio by Lagrangian Multiplier', fontsize=18)
plt.xlabel('Lagrangian Multiplier', fontsize=16)
plt.ylabel('Ratio', fontsize=16)

plt.grid(axis='y', linestyle='--', alpha=0.7)
# Add a reference point to show mean ratio for depth=1 multiangle QAOA
plt.plot(1, red_dot, 'mo', markersize=8, label='Including Infeasible Vanilla λ = 1')

plt.legend(title='QAOA Variant and Depth', fontsize=16,loc='lower right')

# Add horizontal line at ratio=1 to indicate optimal solution quality
plt.axhline(y=1, color='red', linestyle='--', linewidth=2)
plt.tight_layout()
plt.savefig('lagrangian_ratio.pdf', bbox_inches='tight')
plt.show()

#
plt.title('Runtime by Lagrangian Multiplier', fontsize=14)
# Customize the plot

sns.barplot(
    palette='viridis',
    hue='QAOA Implementation',
    y='time_elapsed',
    x='lagrangian_multiplier', 
    data=subset, )


plt.figure(figsize=(12, 6)) # Create a figure for quantum function evaluations by Lagrangian multiplier
plt.figure(figsize=(12, 6))

# Create the boxplot to show distribution
sns.barplot(
    data=subset, 
    x='lagrangian_multiplier', 
    y='quantum_func_evals',
    hue='QAOA Implementation',
    palette='viridis'
)

# Customize the plot
plt.title('Quantum Function Evaluations by Lagrangian Multiplier', fontsize=14)
plt.xlabel('Lagrangian Multiplier', fontsize=12)
plt.ylabel('Number of Function Evaluations', fontsize=12)
plt.legend(title='QAOA Variant and Depth', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('lagrangian_func_evals.pdf', bbox_inches='tight')
plt.show()

# Create a figure for runtime by Lagrangian multiplier

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Group and aggregate the ratio
df = results_lagrange.groupby(['lagrangian_multiplier', 'hamming_dist', 'warm_start'])['ratio'].mean().reset_index()

df.loc[df['warm_start']==False, ['hamming_dist']] = None



# Pivot tables for warm_start = True and False
pivot_true = df.pivot(
    index='lagrangian_multiplier', columns='hamming_dist', values='ratio'
)


plt.figure(figsize=(6, 5))
ax = sns.heatmap(pivot_true, annot=True, fmt=".3f", cmap="Greens")  # <-- Capture the axis as 'ax'

ax.set_title("Average Ratio per combination of WS and Lagrange")
ax.set_xlabel("Hamming Distance")
ax.set_ylabel("Lagrangian Multiplier")

# Fix NaN tick labels on x-axis
xticklabels = ax.get_xticklabels()
new_labels = ['No WS' if str(label.get_text()) == 'nan' else label.get_text() for label in xticklabels]
ax.set_xticklabels(new_labels)

plt.tight_layout()
plt.savefig('heatmap.pdf',bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# --- Data prep ---
df_analysis = results_lagrange.copy()
df_analysis = df_analysis[df_analysis['warm_start']==False]
df_analysis['infeasible_bool'] = df_analysis['infeasible'].astype(str) == 'True'
# Group by config and count
grouped = df_analysis.groupby(['qaoa_variant', 'lagrangian_multiplier',  'infeasible_bool']).size().reset_index(name='count')
print(grouped.value_counts())
# Pivot to wide format
pivot_df = grouped.pivot_table(
    index=['qaoa_variant', 'lagrangian_multiplier', ],
    columns='infeasible_bool',
    values='count',
    fill_value=0
).reset_index()

# Rename for clarity
pivot_df.rename(columns={False: 'feasible', True: 'infeasible'}, inplace=True)

# Calculate percentages
pivot_df['total'] = pivot_df['feasible'] + pivot_df['infeasible']
pivot_df['feasible_pct'] = pivot_df['feasible'] / pivot_df['total'] * 100
pivot_df['infeasible_pct'] = pivot_df['infeasible'] / pivot_df['total'] * 100

# Custom labels for x-axis
pivot_df['label'] = pivot_df.apply(lambda row: f"Lagrangian {row['lagrangian_multiplier']}, {row['qaoa_variant'].capitalize()}", axis=1)

# --- Plotting ---
sns.set_theme(style="whitegrid", context="talk")
colors = {'Feasible': '#3D5B8D', 'Infeasible': '#9D3E42'}

fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(pivot_df))
width = 0.6

# Plot feasible bar base
bars1 = ax.bar(x, pivot_df['feasible_pct'], width, label='Feasible', color=colors['Feasible'])

# Plot infeasible on top
bars2 = ax.bar(x, pivot_df['infeasible_pct'], width, bottom=pivot_df['feasible_pct'], 
               label='Infeasible', color=colors['Infeasible'])

# Add percentage labels
for i, row in pivot_df.iterrows():
    if row['feasible_pct'] > 0:
        ax.text(x[i], row['feasible_pct'] / 2, f"{row['feasible_pct']:.1f}%", ha='center', va='center', fontsize=10, color='white')
    if row['infeasible_pct'] > 0:
        ax.text(x[i], row['feasible_pct'] + row['infeasible_pct'] / 2, f"{row['infeasible_pct']:.1f}%", ha='center', va='center', fontsize=10, color='Black')
# Axes and title
ax.set_xticks(x)
ax.set_xticklabels(pivot_df['label'], rotation=30, ha='right', fontsize=15)
ax.set_ylabel("Percentage of Solutions", fontsize=15)
ax.set_xlabel("QAOA Configuration", fontsize=15)
ax.set_ylim(0, 100)

# Red dashed baseline at 100% (optional)

# Legend and style
ax.legend(title="Solution Type", fontsize=11, title_fontsize=12, loc="lower right")
sns.despine()
plt.tight_layout()
plt.savefig('lagrangian_comparison.pdf', bbox_inches='tight')

plt.show()
